In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pymatgen_diffusion.utils.get_db_data import get_ent_from_db, get_aeccar_from_store, get_deviation_from_optimal_cell_shape
import json
from maggma.stores import MongoStore
from pymatgen_diffusion.neb import get_migration_path as gmp
import copy
import numpy as np
import pymatgen_diffusion.utils.get_db_data
from pymatgen_diffusion.neb import get_migration_path as gmp
from pymatgen_diffusion.neb.full_path_mapper import FullPathMapper as fpm
from crystal_toolkit.helpers.pythreejs_renderer import display_scene
from crystal_toolkit.core.scene import Scene
from crystal_toolkit.components import StructureMoleculeComponent
from crystal_toolkit.components import Spheres, Cubes, Cylinders

with open('.db_info.json') as json_file:
    db_login = json.load(json_file)

elec = MongoStore("js_cathodes", "concat_elec_basf",
                  host=db_login['host'],
                  username=db_login['username'],
                  password=db_login['password'],
                  lu_field="last_updated")

material = MongoStore("js_cathodes", "materials_js",
                  host=db_login['host'],
                  username=db_login['username'],
                  password=db_login['password'],
                  lu_field="last_updated")

tasks = MongoStore("js_cathodes", "tasks",
                    host=db_login['host'],
                    username=db_login['username'],
                    password=db_login['password'],
                    lu_field="last_updated")
elec.connect()
material.connect()
tasks.connect()

In [3]:
#battid = '1292_Li'
#grouped_entries = get_ent_from_db(elec, material, tasks, batt_id=battid, get_aeccar=True)
#inserted_entries = [insert['inserted'][0] for insert in grouped_entries]
#cep = gmp.get_cep_from_group(grouped_entries[0]['base'], grouped_entries[0]['inserted'], working_ion='Li', ltol=0.6, stol=0.8, angle_tol=10)
#struct = cep.base_structure_full_sites
#path = list(cep.get_least_chg_path())

In [4]:
def modify_path(path):
    if path[0][2]['to_jimage'] != (0,0,0):
        path = [i for i in path[::-1]]
        
    start = list(set(path[0][0:2]) & set(path[-1][0:2]))[0]
    last_point = start
    m_path=[]
    for i in range(0,len(path)):
        one_hop = path[i][0:2]
        if one_hop[0] == last_point:
            m_path.append(path[i])
        if one_hop[1] == last_point:
            i_point = path[i][1]
            e_point = path[i][0]

            info = path[i][2]
            new_info = copy.deepcopy(path[i][2])
            new_info['to_jimage'] = tuple([-i for i in info['to_jimage']])
            new_info['ipos'] = info['epos'] - np.array(info['to_jimage'])
            new_info['epos'] = info['ipos'] + np.array(new_info['to_jimage'])
            new_info['ipos_cart'] = info['epos_cart']
            new_info['epos_cart'] = info['ipos_cart']
            new_info.pop('hop')

            new_hop = (i_point, e_point, new_info)
            m_path.append(new_hop)

        last_point = m_path[-1][1]
    
    return m_path

In [5]:
def extract_info(m_path):
    simple = []
    for i in m_path:
        hop = [i[0:2],i[2]['to_jimage']]
        simple.append(hop)
        
    coords = []
    cart_coords = []
    modifier = np.array([0,0,0])
    for i in range(0,len(m_path)):
        hop = m_path[i]
        coords.append([hop[2]['ipos'] + modifier, hop[2]['epos'] + modifier])
        cart_coords.append([hop[2]['ipos_cart'], hop[2]['epos_cart']])
        modifier += np.array(hop[2]['to_jimage'])
        
    return simple, coords, cart_coords

In [6]:
from IPython.display import JSON
from json import JSONEncoder, loads
class MyEncoder(JSONEncoder):
    def default(self, o):
        try:
            return o.as_dict()
        except:
            try:
                return o.__dict__
            except:
                return str(o)
show_json = lambda x : display(JSON(loads(MyEncoder().encode(x))))

In [7]:
def get_extra_scene(pairs, s_radius=0.8, c_radius=0.4): 
    '''
    Takes in position pairs and draw a path
    '''
    extra_scene=[]
    
    [ini_color, final_color]=[[240, 240, 240], [0, 0, 0]]
    div = len(pairs) - 1
    if div == 1:
        return [tuple(ini_color), tuple(final_color)]
    else:
        step_size = [int((final_color[i] - ini_color[i])/(div+1)) for i in range(0,3)]
        rgb_list = [(ini_color[0] + u*step_size[0], ini_color[1] + u*step_size[1], ini_color[2] + u*step_size[2]) for u in range(1, div+1)]
        rgb_list.insert(0, tuple(ini_color))
        rgb_list.append(tuple(final_color))
    rgb_to_html = lambda rgb: '#%02x%02x%02x' % rgb
    html_colors = [rgb_to_html(i) for i in rgb_list]

    
    for i in range(0, len(pairs)):
        if i == 0:
            extra_scene.append(Spheres(positions=[pairs[i][0]], radius=s_radius, color=html_colors[0]))
            extra_scene.append(Spheres(positions=[pairs[i][1]], radius=s_radius, color=html_colors[1]))
            extra_scene.append(Cylinders(positionPairs=[pairs[i]], radius=c_radius, color='black'))
        else:
            extra_scene.append(Spheres(positions=[pairs[i][1]], radius=s_radius, color=html_colors[i+1]))
            extra_scene.append(Cylinders(positionPairs=[pairs[i]], radius=c_radius, color='black'))
    return extra_scene

In [8]:
def get_combined_scene(batt_id, elec, material, tasks, with_migration_specie=False):
    '''
    Obtain a combined scene of the original given strucutre and the migration path.
    '''
    battid = '1292_Li'
    grouped_entries = get_ent_from_db(elec, material, tasks, batt_id=battid, get_aeccar=True)
    cep = gmp.get_cep_from_group(grouped_entries[0]['base'], grouped_entries[0]['inserted'], working_ion='Li', ltol=0.6, stol=0.8, angle_tol=10)
    path = list(cep.get_least_chg_path())
    pairs = extract_info(modify_path(path))[2]
    
    if with_migration_specie == False:
        base_graph = cep.base_struct_entry.structure
    else:
        base_graph = cep.base_structure_full_sites
    smc = StructureMoleculeComponent(base_graph)
    ori_scene = smc.initial_graph.get_scene()
    
    extra_scene = Scene(name='test', contents=get_extra_scene(pairs))
    combined_scene = Scene(name='combine_scene', contents=[ori_scene, extra_scene])
    
    return combined_scene

In [9]:
gcs = get_combined_scene('1292_Li', elec, material, tasks)

/Users/haomingli/uw/pymatgen/pymatgen/core/structure.py:638: UserWarning:

Not all sites have property magmom. Missing values are set to None.



Charge Store


/Users/haomingli/uw/crystaltoolkit/crystal_toolkit/core/mpcomponent.py:65: UserWarning:

No app defined for component StructureMoleculeComponent, callbacks cannot be created. Please register app using MPComponent.register_app(app).

/Users/haomingli/uw/crystaltoolkit/crystal_toolkit/core/mpcomponent.py:72: UserWarning:

No cache is defined for component StructureMoleculeComponent, performance of app may be degraded. Please register cache using MPComponent.register_cache(cache).



In [10]:
display_scene(gcs)

/Users/haomingli/uw/crystaltoolkit/crystal_toolkit/helpers/pythreejs_renderer.py:121: UserWarning:

Primitive type convex has not been implemented for this renderer.



Renderer(background='white', camera=OrthographicCamera(bottom=-20.200857648851184, left=-20.200857648851184, n…